In [13]:
from __future__ import print_function
from __future__ import division
import numpy as np 
import pandas as pd 

import torch
import sys
import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from sklearn import cross_validation
from sklearn import metrics
from sklearn.metrics import roc_auc_score, log_loss, roc_auc_score, roc_curve, auc
from sklearn.cross_validation import StratifiedKFold, ShuffleSplit, cross_val_score, train_test_split
from torch.utils.data import TensorDataset, DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [14]:
print('__Python VERSION:', sys.version)
print('__pyTorch VERSION:', torch.__version__)

__Python VERSION: 3.6.3 |Anaconda, Inc.| (default, Nov  8 2017, 15:10:56) [MSC v.1900 64 bit (AMD64)]
__pyTorch VERSION: 0.3.0b0+591e73e


In [15]:
use_cuda = torch.cuda.is_available()
if use_cuda:
    num_workers = 0 # for windows version of PyTorch which does not share GPU tensors
else:
    num_workers = 4

global_seed=999
# Data params
TARGET_VAR = 'target'

# BASE_FOLDER = '../input/'
BASE_FOLDER = 'D:/dataset/kaggle_Iceberg'

batch_size = 256

validationRatio = 0.11
LR = 0.0005
MOMENTUM = 0.95
global_seed=999

In [16]:
import Iceberg_dataset
import iceberg_classfier

In [17]:
import numpy
import numpy as np
import random
from datetime import datetime
from sklearn.utils import shuffle

def fixSeed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if use_cuda:
        torch.cuda.manual_seed(seed)

In [18]:
import math

def savePred(df_pred, val_score):
    csv_path = str(val_score) + '_sample_submission.csv'
    df_pred.to_csv(csv_path, columns=('id', 'is_iceberg'), index=None)
    print(csv_path)

In [19]:
fixSeed(global_seed)

In [20]:
import models
for i in range (0 , 30):
    model = models.SimpleNet()
    # model = ResNetLike(BasicBlock, [1, 3, 3, 1], num_channels=2, num_classes=1)
    print ("Ensamble number:" + str(i))
    data, full_img = Iceberg_dataset.readSuffleData(global_seed,BASE_FOLDER)
    train_loader, val_loader, train_ds, val_ds = Iceberg_dataset.getTrainValLoaders(data,full_img,batch_size,num_workers,validationRatio)
    # train_loader, val_loader, train_ds, val_ds = getCustomTrainValLoaders()
    model, val_result,train_result=iceberg_classfier.generateSingleModel(model,train_loader, val_loader, train_ds, val_ds,LR,55)

Ensamble number:0


AssertionError: Torch not compiled with CUDA enabled

In [28]:
from Iceberg_dataset import readSuffleData,getTrainValLoaders,getCustomTrainValLoaders,fixSeed
model = models.SimpleNet()   
data, full_img = Iceberg_dataset.readSuffleData(global_seed,BASE_FOLDER)
train_loader, val_loader, train_ds, val_ds = getCustomTrainValLoaders(data,full_img,batch_size,num_workers,validationRatio,global_seed)

In [34]:
for i, row_data in enumerate(train_loader, 1):
    img = row_data['image']
    label = row_data['labels']
    img, label = Variable(img.cuda(async=True)), Variable(label.cuda(async=True))

In [36]:

from Iceberg_dataset import readSuffleData,getTrainValLoaders,getCustomTrainValLoaders,fixSeed
model = models.SimpleNet()   
data, full_img = Iceberg_dataset.readSuffleData(global_seed,BASE_FOLDER)
train_loader_a, val_loader_a, train_ds, val_ds = getTrainValLoaders(data,full_img,batch_size,num_workers,validationRatio)

In [33]:
import math

def savePred(df_pred, val_score):
    csv_path = str(val_score) + '_sample_submission.csv'
    df_pred.to_csv(csv_path, columns=('id', 'is_iceberg'), index=None)
    print(csv_path)

In [41]:
for i, row_data in enumerate(train_loader_a, 1):
    img,label = row_data
    print(img)
    #img, label = Variable(img.cuda(async=True)), Variable(label.cuda(async=True))


( 0 , 0 ,.,.) = 
 -19.4066 -18.1456 -18.8801  ...  -19.6863 -18.6322 -20.4167
 -19.1393 -18.2638 -17.5778  ...  -20.7269 -18.5090 -19.5474
 -19.4066 -18.2638 -17.1487  ...  -21.5548 -18.7572 -20.5705
            ...               ⋱              ...            
 -17.6892 -19.0096 -23.4900  ...  -23.4938 -24.9055 -23.7138
 -19.8245 -20.2626 -20.1142  ...  -19.6872 -18.7581 -20.1181
 -20.1140 -18.6292 -17.6893  ...  -19.4114 -18.3884 -20.1181

( 0 , 1 ,.,.) = 
 -27.9260 -29.9552 -33.9467  ...  -27.5753 -27.5754 -27.9300
 -27.9260 -28.2957 -30.4248  ...  -28.2996 -27.5754 -27.5754
 -26.9029 -28.6818 -31.4479  ...  -27.5754 -27.9300 -27.9301
            ...               ⋱              ...            
 -26.9038 -27.5723 -29.9561  ...  -26.5919 -28.3005 -26.5920
 -27.2316 -27.5723 -26.9039  ...  -26.9077 -27.9308 -29.0907
 -27.9269 -25.7039 -24.6498  ...  -29.9599 -29.5145 -29.0907
      ⋮  

( 1 , 0 ,.,.) = 
 -24.6193 -18.5988 -18.5988  ...  -18.7211 -18.7211 -18.0351
 -21.0196 -23.5184 -2

In [ ]:
import iceberg_classfier
df_pred = iceberg_classfier.testModel(model)
savePred(df_pred, val_result)